In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed May 29 15:40:50 2019

@author: soumyadeep
"""

#****************************************
# ORBITAL PROJECTED BAND STRUCTURE      *
#****************************************

#**********************************************************************
#               PART-1                                                *
# MAKING OF DATA FILE FOR PROJECTIONS                                 *
#**********************************************************************

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from matplotlib import cm
from matplotlib.mlab import griddata
from matplotlib import rc
import matplotlib.ticker as ticker
import matplotlib.lines
import csv
from collections import OrderedDict
import xml.etree.ElementTree as ET
import numpy as np
import itertools
import sys, os
import glob   # for stacking csv files
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)


NAME = 'LiFeAs-0GPa'
EXAMPLE_DIR = '/media/soumyadeep/SOUMYA2/Python/111 Pressure/LiFeAs-0GPa/10_WF/OPBS'
SAVE_DIR='/media/soumyadeep/SOUMYA2/Python/111 Pressure/LiFeAs-0GPa/10_WF'

# TO REMOVE OLDER FILES, OTHERWISE COMMENT OUT
#os.remove(EXAMPLE_DIR+'/data.csv')
#os.remove(EXAMPLE_DIR+'/energy.csv')
#os.remove(EXAMPLE_DIR+'/k_points.csv')
#os.remove(EXAMPLE_DIR+'/projections.csv')

# TARGET DIRECTORY FOR SAVING PROJECTION FILES
#os.mkdir(EXAMPLE_DIR+'/proj_info')

# PARSING .xml FILE FROM EXAMPLE DIR
tree = ET.parse(EXAMPLE_DIR+'/atomic_proj.xml')
root = tree.getroot()

# READING OF .xml FILE
header = root.find('HEADER')
nbnd  = int(header.find('NUMBER_OF_BANDS').text.strip())
nkpts = int(header.find('NUMBER_OF_K-POINTS').text.strip())
natwfc= int(header.find('NUMBER_OF_ATOMIC_WFC').text.strip())
nspin = int(header.find('NUMBER_OF_SPIN_COMPONENTS').text.strip())

# MATERIAL SPECIFIC INPUT
sspin = 1 #selected spin component.
nproj=16  #number of projections 
# decrease wave function number by one unit e.g. for wfc10 write wfc9
wf_number=[8,9,10,11,12,21,22,23,24,25,31,32,33,35,36,37]    
Efermi=8.6793    # Fermi energy in eV
print(nbnd,nkpts,natwfc,nspin)



# START OF PROGRAM
print('PART-1 running ........')

d = root.find('K-POINTS')
kpts = np.array(d.text.strip().split(),dtype=np.float)
kpts = kpts.reshape([nkpts,3])
for i in range(len(kpts)):
    a=kpts[i][0]       # kx
    b=kpts[i][1]       # ky
    c=kpts[i][2]       # kz
    if i==0:
        k=0.0
    else:
        k+=np.sqrt((kpts[i][0]-kpts[i-1][0])**2+(kpts[i][1]-kpts[i-1][1])**2+(kpts[i][2]-kpts[i-1][2])**2)
    with open(EXAMPLE_DIR+"/k_points.csv","a") as output_file:
      output_fileWriter=csv.writer(output_file)
      output_fileWriter.writerow([a,b,c,k])
    output_file.close()

print('K-POINT CARD GENERATED')




d = root.find('WEIGHT_OF_K-POINTS')
wgth = np.array(d.text.strip().split(),dtype=np.float)


eigs = root.find('EIGENVALUES')
es = []
for i in range(nkpts):
    # set estr, the string for parsing eigenvalues
    if nspin > 1:
        estr = 'EIG.'+str(sspin)
    else:
        estr = 'EIG'
        
    e = np.array(eigs.find('K-POINT.'+str(i+1)).find(estr).text.strip().split(),dtype=np.float)
    es.append(e)
for i in range(nkpts):
    for j in range(nbnd):
        #print(es[i][j])
        with open(EXAMPLE_DIR+"/energy.csv","a") as output_file:
            output_fileWriter=csv.writer(output_file)
            output_fileWriter.writerow([es[i][j]])
        output_file.close()
    
print('EIGENVALUE CARD GENERATED')


p = root.find('PROJECTIONS')

wfcs=[]
for nk in range(nkpts):
    k = p.find('K-POINT.'+str(nk+1))
    wfc = np.zeros([natwfc,nbnd],dtype=np.complex)
    #print(wfc)
    
    if nspin > 1:
        k = k.find('SPIN.'+str(sspin))
    
    for a in range(natwfc):
        d = k.find('ATMWFC.'+str(a+1)).text.strip().split('\n')
        #print(d)
    
        for i in range(nbnd):
            wfc[a,i]=np.complex(*[float(x) for x in d[i].split(',')])
    wfcs.append(wfc)


rwfcs = np.swapaxes(wfcs,1,2)
#print(rwfcs[99][40][60])

#wfcs[:,[1, 2]] = wfcs[:,[2, 1]]
#print(rwfcs)


for i in range(nkpts):
    for j in range(nbnd):
        for k in range(natwfc): 
            proj=rwfcs[i][j][k].flat[0]       
            if k in (wf_number):
            #if (24 <= k <= 28 or 37 <= k <= 41):
            #if (2 <= k <= 7):
                #print(k)
                with open(EXAMPLE_DIR+"/projections.csv","a") as output_file:
                    output_fileWriter=csv.writer(output_file)
                    output_fileWriter.writerow([i,j,k,proj.real,proj.imag,abs(proj)])
                output_file.close()

print('PROJECTION CARD GENERATED')


kx,ky,kz,k = np.loadtxt(EXAMPLE_DIR+'/k_points.csv', delimiter=',', unpack=True)
E = np.loadtxt(EXAMPLE_DIR+'/energy.csv', delimiter=',', unpack=True)
ii,jj,kk,Pr,Pi,Pa = np.loadtxt(EXAMPLE_DIR+'/projections.csv', delimiter=',', unpack=True)


#print(len(k),len(E),len(Pa))
for x in range(nkpts):
    for y in range(nbnd):
        y+=nbnd*x
        for z in range(nproj):
            z+=nproj*y
            #print(x,y,z)
            with open(EXAMPLE_DIR+"/data.csv","a") as output_file:
                output_fileWriter=csv.writer(output_file)
                output_fileWriter.writerow([k[x],E[y],Pa[z]])
            output_file.close()

print('DATA CARD GENERATED')            
print('PART-1 completed ........')


#**********************************************************************
#               PART-2                                                *
# SEPARATE PROJECTIONS FOR SCATTER PLOT                               *
# change if wave function number is changed                           *
#**********************************************************************
print('PART-2 running ........')

for band_number in range(0,nbnd,1):
    #print(band_number)
    k, E, Pa = np.loadtxt(EXAMPLE_DIR+'/data.csv', delimiter=',', unpack=True)
    E=(E*13.6)-Efermi
    for j in range(0,len(k),nbnd*nproj):   
        i=j+nproj*(band_number-1)   
        #print(k[i],E[i],Pa[i]+Pa[i+5])
        Ptot=Pa[i]+Pa[i+1]+Pa[i+2]+Pa[i+3]+Pa[i+4]+Pa[i+5]+Pa[i+6]+Pa[i+7]+Pa[i+8]+Pa[i+9]+Pa[i+10]+Pa[i+11]+Pa[i+12]+Pa[i+13]+Pa[i+14]+Pa[i+15]
        with open(EXAMPLE_DIR+"/proj_info/proj_" + str(band_number+1) + ".csv","a") as output_file:
            output_fileWriter=csv.writer(output_file) 
            output_fileWriter.writerow([k[i],E[i],
                                    (Pa[i]+Pa[i+5])/Ptot,
                                    (Pa[i+1]+Pa[i+6])/Ptot,
                                    (Pa[i+2]+Pa[i+7])/Ptot,
                                    (Pa[i+3]+Pa[i+8])/Ptot,
                                    (Pa[i+4]+Pa[i+9])/Ptot,
                                    (Pa[i+10]+Pa[i+13])/Ptot,
                                    (Pa[i+11]+Pa[i+14])/Ptot,
                                    (Pa[i+12]+Pa[i+15])/Ptot])
            output_fileWriter=csv.writer(output_file)

print('PLOTTING CARD GENERATED') 
print('PART-2 completed ........')

#**********************************************************************
#               PART-3                                                *
# PLOTTING OF ORBITAL PROJECTED BAND STRCUTURE                        *
# change if wave function weight is changed                           *
#**********************************************************************
print('PART-3 running ........')
print('PLOTTING OPBS')
 
path = EXAMPLE_DIR+'/proj_info/proj_*.csv'

fig,ax = plt.subplots()
for fname in glob.glob(path):
    #print(fname)    
    k, E, d_z2, d_zx, d_zy, d_x2y2, d_xy, p_z, p_x, p_y = np.loadtxt(fname, delimiter=',', unpack=True)
    k = k/max(k)
    plt.plot(k, E+0.05, c='k',lw=0.5,alpha=0.4)    
    p1=plt.scatter(k, E+0.05, s=(15*d_z2)**2, c='r', marker='.',alpha=0.5,label='$3d_{z^2}$')
    p2=plt.scatter(k, E+0.05, s=(15*d_zx)**2, c='navy', marker='.',alpha=1.0,label='$3d_{zx}$')
    p3=plt.scatter(k, E+0.05, s=(15*d_zy)**2, c='magenta', marker='.',alpha=0.5,label='$3d_{yz}$')
    p4=plt.scatter(k, E+0.05, s=(15*d_x2y2)**2, c='orange', marker='.',alpha=0.4,label='$3d_{x^2-y^2}$')
    p5=plt.scatter(k, E+0.05, s=(15*d_xy)**2, c='green', marker='.',alpha=0.8,label='$3d_{xy}$')
    p6=plt.scatter(k, E+0.05, s=(15*p_z)**2, c='cyan', marker='.',alpha=0.5,label='$4p_{z}$')
    p7=plt.scatter(k, E+0.05, s=(15*p_x)**2, c='dimgray', marker='.',alpha=0.5,label='$4p_{x}$')
    p8=plt.scatter(k, E+0.05, s=(15*p_y)**2, c='olive', marker='.',alpha=1.0,label='$4p_{y}$')


# AXES PROPERTIES
plt.ylim(-1.0,1.0)
plt.xlim(min(k), max(k))
plt.axhline(y=0.0, color='k',ls='--',lw=1.0)
plt.axvline(x=0.333,color='k', ls='--',lw=1.0)
plt.axvline(x=0.666,color='k', ls='--',lw=1.0)
#plt.axvline(x=0.434,color='k', ls='--',lw=1.0)
#plt.axvline(x=0.565,color='k', ls='--',lw=1.0)
#plt.axvline(x=0.738,color='k', ls='--',lw=1.0)
#plt.axvline(x=0.868,color='k', ls='--',lw=1.0)
#plt.xticks([0.0,0.172,0.303,0.434,0.565,0.738,0.868,1.0], ['$\Gamma$', 'X', 'M', '$\Gamma$', 'Z', 'R', 'A', 'Z'],fontsize=18)
plt.xticks([0.0,0.333,0.666,1.0], ['$\Gamma$', 'M', '$\Gamma$', 'M'],fontsize=18)
plt.ylabel(r'$\rm E-E_{F}(eV)$',fontsize=18,fontweight='bold')
plt.tick_params(axis='y', which='major', length=5.0, direction='out', bottom=False, top=False, left=True, right=False, labelsize = 12.0)
plt.tick_params(axis='y', which='minor', length=3.0, direction='out', bottom=False, top=False, left=True, right=False, labelsize = 12.0)
#ml = MultipleLocator(2)
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax.yaxis.set_minor_locator(MultipleLocator(2))
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"


# WRITE Ef IN RIGHT SIDE
plt.annotate(r'$\rm E_{F}$', xy=(0,0), xytext=(1.01,-0.05),fontsize=18,weight="bold")

       

# CREATE SOME SPACE IN FOUR SIDES OF PLOT
plt.subplots_adjust(top=0.85, left=0.12, right=0.94, bottom=0.1) 

# set up legends
lgnd=plt.legend((p1,p2,p3,p4,p5,p6,p7,p8), 
    ('$3d_{z^2}$','$3d_{xz}$','$3d_{yz}$','$3d_{x^2-y^2}$','$3d_{xy}$','$4p_{z}$','$4p_{x}$','$4p_{y}$'),
    loc='lower center', bbox_to_anchor=(0.5, 1.02), markerscale=6,
    fancybox=True, shadow=False, ncol=4 ,fontsize=12, borderaxespad=0.1) 

#change the marker size manually for both lines
lgnd.legendHandles[0]._sizes = [1000]
lgnd.legendHandles[1]._sizes = [1000]
lgnd.legendHandles[2]._sizes = [1000]
lgnd.legendHandles[3]._sizes = [1000]
lgnd.legendHandles[4]._sizes = [1000]
lgnd.legendHandles[5]._sizes = [1000]
lgnd.legendHandles[6]._sizes = [1000]
lgnd.legendHandles[7]._sizes = [1000]

# SAVING PART
#plt.tight_layout()
fig.set_size_inches(7,5)
plt.savefig('/media/soumyadeep/SOUMYA2/Python/test_fig.pdf', aspect='auto', dpi=4500, alpha=0.1)
#plt.savefig(SAVE_DIR+"/"+NAME+"_OPBS.pdf",bbox_inches='tight', dpi=4500)  
#plt.savefig(SAVE_DIR+"/"+NAME+"_OPBS.eps",bbox_inches='tight', dpi=4500)    
#plt.savefig(SAVE_DIR+"/"+NAME+"_OPBS.jpeg",bbox_inches='tight', dpi=1500)  
plt.show() 



print('PART-3 completed ........')
print('ALL DONE')

